Load Libraries and Data

In [20]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

X_testDataPath = "./dog-breed-identification/test/"
X_trainDataPath = "./dog-breed-identification/train/"
y_DataPath = './dog-breed-identification/labels.csv'

data = pd.read_csv(y_DataPath)

xt, xtr, yt, ytr = [],[],[],[]

for _ , example in data.iterrows():
    if os.path.exists(X_testDataPath + example["id"] + ".jpg"):
        xt.append(example["id"])
        yt.append(example["breed"])
    else:
        xtr.append(example["id"])
        ytr.append(example["breed"])

X_train, y_train, X_test, y_test = pd.DataFrame(xtr), pd.DataFrame(ytr), pd.DataFrame(xt), pd.DataFrame(yt)

                                      0
0      000bec180eb18c7604dcecc8fe0dba07
1      001513dfcb2ffafc82cccf4d8bbaba97
2      001cdf01b096e06d78e9e5112d419397
3      00214f311d5d2247d5dfe4fe24b2303d
4      0021f9ceb3235effd7fcde7f7538ed62
...                                 ...
10217  ffd25009d635cfd16e793503ac5edef0
10218  ffd3f636f7f379c51ba3648a9ff8254f
10219  ffe2ca6c940cddfee68fa3cc6c63213f
10220  ffe5f6d8e2bff356e9482a80a6e29aac
10221  fff43b07992508bc822f33d8ffd902ae

[10222 rows x 1 columns]
